
# Data Pre-Processing
Note: This dataset has already been scaled

In [1]:
# Impord dependencies 

import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
!ls data

Test
Train
voice.csv
voice.md


In [4]:
# Read test and training data

# Training Data Paths
X_training_data = os.path.join("data", "Train", "X_train.txt")
y_training_data = os.path.join("data", "Train", "y_train.txt")

# Testing Data Paths
X_testing_data = os.path.join("data", "Test", "X_test.txt")
y_testing_data = os.path.join("data", "Test", "y_test.txt")

In [5]:
# Read the training data into a dataframe
X_train_df = pd.read_csv(X_training_data, delimiter=" ", skiprows=1, header=None)
X_train_df.head()

# Convert the dataframe to a numpy array for Keras
X_train = X_train_df.values

# Read in the training labels as a dataframe
y_train_df = pd.read_csv(y_training_data)


In [31]:
y_train_df.head()
# 5 is STANDING

,5
0,5
1,5
2,5
3,5
4,5


In [9]:
# One-hot encode the integer labels
# 1 WALKING           
# 2 WALKING_UPSTAIRS  
# 3 WALKING_DOWNSTAIRS
# 4 SITTING           
# 5 STANDING          
# 6 LAYING            
# 7 STAND_TO_SIT      
# 8 SIT_TO_STAND      
# 9 SIT_TO_LIE        
# 10 LIE_TO_SIT        
# 11 STAND_TO_LIE      
# 12 LIE_TO_STAND      

y_train = to_categorical(y_train_df)
y_train[:10]

# Read the testing data
X_test_df = pd.read_csv(X_testing_data, delimiter=" ", skiprows=1, header=None)
X_test = X_test_df.values

# Read the testing labels
y_test_df = pd.read_csv(y_testing_data)

# One-hot encode the integer labels
y_test = to_categorical(y_test_df)

# Deep Neural Network

In [10]:
# Create an empty sequential model
model = Sequential()

# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

# Add a second hidden layer
model.add(Dense(100, activation='relu'))

In [11]:
# The output layer has 13 columns that are one-hot encoded
y_train.shape

(7766, 13)

In [14]:
# Add output layer
# This time 'linear'
model.add(Dense(y_train.shape[1], activation="linear"))

In [15]:
# Compile the model using mean squared error for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])

In [16]:
# Use the training data to fit (train) the model
model.fit(
    X_train,
    y_train,
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
 - 2s - loss: 0.0326 - acc: 0.7919
Epoch 2/50
 - 1s - loss: 0.0135 - acc: 0.9187
Epoch 3/50
 - 1s - loss: 0.0096 - acc: 0.9410
Epoch 4/50
 - 1s - loss: 0.0075 - acc: 0.9558
Epoch 5/50
 - 1s - loss: 0.0066 - acc: 0.9597
Epoch 6/50
 - 1s - loss: 0.0062 - acc: 0.9585
Epoch 7/50
 - 1s - loss: 0.0052 - acc: 0.9665
Epoch 8/50
 - 1s - loss: 0.0058 - acc: 0.9648
Epoch 9/50
 - 1s - loss: 0.0051 - acc: 0.9676
Epoch 10/50
 - 1s - loss: 0.0044 - acc: 0.9732
Epoch 11/50
 - 1s - loss: 0.0043 - acc: 0.9735
Epoch 12/50
 - 1s - loss: 0.0039 - acc: 0.9759
Epoch 13/50
 - 1s - loss: 0.0043 - acc: 0.9727
Epoch 14/50
 - 1s - loss: 0.0036 - acc: 0.9763
Epoch 15/50
 - 1s - loss: 0.0035 - acc: 0.9780
Epoch 16/50
 - 1s - loss: 0.0039 - acc: 0.9760
Epoch 17/50
 - 1s - loss: 0.0035 - acc: 0.9763
Epoch 18/50
 - 1s - loss: 0.0034 - acc: 0.9791
Epoch 19/50
 - 1s - loss: 0.0033 - acc: 0.9791
Epoch 20/50
 - 1s - loss: 0.0028 - acc: 0.9824
Epoch 21/50
 - 1s - loss: 0.0030 - acc: 0.9798
Epoch 22/50
 - 1s - lo

# Save the Trained Model

In [17]:
# Save the model
model.save("smartphone_trained.h5")

# Evaluate the Model

In [18]:
# Load the model
from keras.models import load_model
model = load_model("smartphone_trained.h5")

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               56200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 13)                1313      
Total params: 67,613
Trainable params: 67,613
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Evaluate the model using the training data 
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.008132264866977277, Accuracy: 0.9319835495096488


In [51]:
# Grab just one data point to test with
test = np.expand_dims(X_test[0], axis=0)
test.shape

(1, 561)

In [52]:
# Make a prediction. The result should be 5 - STANDING
print(f"Predicted class: {model.predict_classes(test)}")
print(f"Actual class: {y_test_df.iloc[:1]}")

Predicted class: [5]
Actual class:    5
0  5
